In [2]:
import json
import time
from collections import Counter
import numpy as np
import random
import heapq
import math
import os

# Word2Vec using Negative Sampling

reference: https://arxiv.org/pdf/1310.4546.pdf

## Build Index and Frequency Map

In [3]:
def get_word_map_from_file(file_name, threshold):
    """
    Build dictionary and count frequency for vocabulary in file.

    Arguments:
    file_name -- name of the training sample file.
    threshold -- lower bound of frequency of words to be included

    Returns:
    word2idx_map -- dictionary of {word : index}
    idx2freq_map -- list
    idx2word_map - list

    """
    with open(file_name, 'rb') as f:
        # count the frequency of words
        count = Counter()
        for line in f.readlines():
            for w in line.decode('utf-8').strip().split():
                count.update(w)
                
    # sort the words by its frequency in ascending order
    word_count_list = list(filter(lambda x: x[1] >= threshold, count.most_common()))
    
    # dict {word : index}
    word2idx_map = {w: idx for idx, (w, _) in enumerate(word_count_list)}
    
    # list
    idx2freq_map = list(map(lambda x : x[1], word_count_list))
    
    # list
    idx2word_map = list(map(lambda x : x[0], word_count_list))
    return word2idx_map, idx2freq_map, idx2word_map

In [4]:
word2idx_map, idx2freq_map, idx2word_map = get_word_map_from_file('poem.txt', 5)

## Skip-gram model

$$\mathcal{L} = \sum_t \sum_{-c \le j \le c, j \ne 0} \mathcal{L}(w_{t+j}, w_j)$$

## Objective of Skip-gram with Negative Sampling

$$P(w_{context} \mid w_{target}) = \sigma(u_{w_{context}}^T v_{w_{target}}) $$

$$P_{neg}(w_i) = \frac{F(w_i)^{\frac{3}{4}}}{\sum_i F(w_i)^{\frac{3}{4}}}\tag{1}$$

$$ \mathcal{L}(w_{out}, w_{in}) = -log(P(w_{out} \mid w_{in})) - \sum_{j=1}^{k} \mathbb {E}_{w_j \backsim  P_{neg}(w_j)} log(1-P(w_j\mid w_{in})) $$

## Subsampling

$$ P(w_i) = (\sqrt{\frac {F(w_i)}{0.001}} + 1)⋅\frac {0.001}{F(w_i)}$$

In [5]:
def create_sampling_table(idx2freq_map, power=0.75, sample=1e-3):
    """
    Build unigram table for vocab.

    Arguments:
    power -- the power to resample vocab frequency

    Returns:
    neg_table -- list of index with resampled frequency
    keep_prob -- numpy array of shape (n_words, )

    """
    neg_freq_list = list(map(lambda x: (x[0], int(x[1] ** power)),
                             enumerate(idx2freq_map)))
    table_size = sum([x[1] for x in neg_freq_list])
    neg_table = np.zeros(table_size).astype(int)
    offset = 0
    for word, freq in neg_freq_list:
        neg_table[offset : offset + freq] = word
        offset += freq

    assert(offset == table_size)
    
    z = np.array(idx2freq_map) / (sample * sum(idx2freq_map))
    keep_prob = (np.sqrt(z) + 1) / z
    
    return neg_table, keep_prob

def init_parameters(vocab_size, embed_size, use_biases=True, init='zero'):
    '''
    Initialize parameters for NN model.

    Argumemts:
    vocab_size -- count of distinct input vocab.
    embed_size -- the vector length to embed word to.
    use_biases -- whether to use the bias term in model.
    init --

    Returns:
    parameters -- dict of model parameters.
                  parameters['embeddings'] -- (embed_size, vocab_size) matrix.
                  parameters['weights'] -- (vocab_size, embed_size) matrix.
                  (optional)
                  parameters['biases'] -- (vocab_size, 1) matrix.
    '''
    parameters = {}
    parameters['embeddings'] = np.random.randn(embed_size, vocab_size) / np.sqrt(vocab_size)
    if init == 'zero':
        parameters['weights'] = np.zeros((vocab_size, embed_size))
    elif init == 'xavier':
        parameters['weights'] = np.random.randn(vocab_size, embed_size) / np.sqrt(embed_size)
    if use_biases:
        parameters['biases'] = np.zeros((vocab_size, 1))
    
    return parameters

def sigmoid(logits):
    return 1 / (1 + np.exp(-logits))

def forward_prop(parameters):
    '''
    Get probabilistic predictions of model.

    Arguments:
    parameters -- dict of model parameters to train.

    Returns:
    pred -- numpy array of shape (n_samples, 1).

    '''
    preds = sigmoid(np.dot(parameters['weights'], parameters['embeddings']) +
                    (parameters['biases'] if 'baises' in parameters else 0))
    
    return preds

def back_prop(preds, labels, parameters):
    '''
    Get model's gradients.

    Arguments:
    preds -- (n_samples, 1) vector of probabilistic predictions.
    labels -- (n_samples, 1) vector of labels.
    parameters -- dictionary of model parameters to train.

    Returns:
    grads -- dictionary of gradients
             grads['embeddings'] -- (embed_size, 1) vector.
             grads['weights'] -- (n_samples, embed_size) vector.
             grads['biases'] -- (n_samples, 1) vector.
    
    '''
    grads = {}
    ds = preds - labels # (n_samples, 1) vector
    grads['embeddings'] = np.dot(parameters['weights'].T, ds)
    grads['weights'] = np.dot(ds, parameters['embeddings'].T)
    if 'biases' in parameters:
        grads['biases'] = ds
    return grads

## Optimizer

$$g_t = \nabla_\theta \mathcal{L}(\theta_{t-1})$$

- ### Gradient Descent
$$\theta_t = \theta_{t - 1} - \alpha g_t$$

- ### Adam
reference: https://arxiv.org/pdf/1412.6980v8.pdf
$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$
$$\widehat{m}_t = \frac{m_t}{1 - \beta_1^{t}}$$
$$\widehat{v}_t = \frac{v_t}{1 - \beta_2^{t}}$$
$$\theta_t = \theta_{t-1} - \alpha \frac{\widehat{m}_t}{\sqrt{\widehat{v}_t} + \epsilon}$$

In [6]:
def get_proj(parameters, word_pairs):
    '''
    find the corresponding projection of word vectors of input word pairs.

    Arguments:
    parameters -- dict of model parameters.
    word_pairs -- tuple (input_word, target_words)
                  input_word -- (1, 1) vector of input word index.
                  target_words -- (n_samples, 1) vector of output word index.
                  (due to negative sampling.)

    Returns:
    parameters_proj -- dict of projected parameters
                       parameters_proj['embeddings'] -- numpy array of shape (embed_size, 1).
                       parameters_proj['weights'] -- numpy array of shape (n_samples, embed_size).
                       (optional) parameters_proj['biases'] -- numpy array of shape (n_samples, 1).
    '''
    (input_word, target_words) = word_pairs
    parameters_proj = {}
    parameters_proj['embeddings'] = parameters['embeddings'][:, input_word]
    parameters_proj['weights'] = parameters['weights'][target_words, :]
    if 'biases' in parameters:
        parameters_proj['biases'] = parameters['biases'][target_words, :]
        
    return parameters_proj
        

def set_proj(parameters, parameters_proj, word_pairs):
    '''
    set the projection back to original parameters dict.

    Arguments:
    parameters -- dict of model parameters.
    parameters_proj -- dict of projected model parameters.

    '''
    (input_word, target_words) = word_pairs
    parameters['embeddings'][:, input_word] = parameters_proj['embeddings']
    parameters['weights'][target_words, :] = parameters_proj['weights']
    if 'biases' in parameters:
        parameters['biases'][target_words, :] = parameters_proj['biases']

class GradientDescentOptimzer:
    def __init__(self, alpha):
        self.alpha = alpha
        
    def update_proj(self, parameters, grads_proj, word_pairs):
        '''
        Arguments:
        parameters -- dictionary
        grads_proj -- dictionary
        word_pairs -- tuple
        '''
        parameters_proj = get_proj(parameters, word_pairs)
        
        for key in parameters_proj.keys():
            parameters_proj[key] -= self.alpha * grads_proj[key]
            
        set_proj(parameters, parameters_proj, word_pairs)

        
class Adam:
    def __init__(self, alpha, parameters, beta1=0.9, beta2=0.999, epsilon=1e-8, decay=0.0):
        self.m = {}
        self.v = {}
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.decay = decay
        self.t = 1
        for key, param in parameters.items():
            self.m[key] = np.zeros_like(param)
            self.v[key] = np.zeros_like(param)
        
    def update_proj(self, parameters, grads_proj, word_pairs):
        '''
        Arguments:
        parameters -- dictionary
        grads_proj -- dictionary
        word_pairs -- tuple
        '''
        m_proj = get_proj(self.m, word_pairs)
        v_proj = get_proj(self.v, word_pairs)
        parameters_proj = get_proj(parameters, word_pairs)
        
        lr = self.alpha * (np.sqrt(1. - np.power(self.beta2, self.t)) /
             (1. - np.power(self.beta1, self.t)))
        if self.decay > 0:
            lr /= 1 + self.decay * (self.t - 1)
        
        for key in parameters.keys():
            m_proj[key] = self.beta1 * m_proj[key] + (1. - self.beta1) * grads_proj[key]
            v_proj[key] = self.beta2 * v_proj[key] + (1. - self.beta2) * np.square(grads_proj[key])
            parameters_proj[key] -= lr * m_proj[key] / (np.sqrt(v_proj[key]) + self.epsilon)
        
        set_proj(self.m, m_proj, word_pairs)
        set_proj(self.v, v_proj, word_pairs)
        set_proj(parameters, parameters_proj, word_pairs)
        self.t += 1
        

In [7]:
def sentence2index(line, word2idx_map, keep_prob):
    sen = []
    for w_raw in line.decode("utf-8").strip().split():
        # get index from dict
        w_idx = word2idx_map.get(w_raw, -1)
        # word not in dic (frequency < threshold)
        if w_idx == -1:
            continue
        # randomly subsamples word due to its freq
        if keep_prob[w_idx] < random.random():
            continue
        sen.append(w_idx)
        
    return sen


def negative_sampling(context, sen_set, k, neg_table):
    target_words = set()

    # positive example
    # pred(input_word, context) should be 1
    
    while len(target_words) < k:
        # out of sentence negative example from table
        # pred(input_word, neg) should be 0
        while True:
            neg_word = np.random.choice(neg_table)
            if neg_word not in sen_set:
                break
        target_words.add(neg_word)
    
    #labels = np.zeros((k + 1, 1))
    #labels[0] = 1.0
    
    return [context] + list(target_words), np.eye(k + 1)[:, [0]]

In [8]:
def save_embedding_to_file(model_name, data, epoch):
    '''
    Arguments:
    model_name -- string
    data -- numpy array
    epoch -- int
    '''
    path = 'output/' + model_name
    if not os.path.isdir(path):
        os.makedirs(path)
    file_name = path + "/epoch%s.json" % epoch

    with open(file_name, "w") as f:
        f.write(json.dumps(data.tolist(), indent=4))
        
def load_embedding_from_file(file_name):
    '''

    Arguments:
    file_name -- the json file name saving results.

    Returns:
    '''
    with open(file_name, "r") as f:
        embeddings = np.array(json.loads("".join(f.readlines())))
    return embeddings

def train_embedding(model_name, file_name, word_map=None,
    embed_size=30, window=2,
    use_biases=False,
    optimizer='gd',
    learning_rate=0.025,
    sample=1e-3,
    negative_samples=15,
    n_epochs=2, 
    verbose=1,
    init='zero'):

    start_time = time.time()
    if not word_map:
        print('Build word map from file:', file_name)
        word2idx_map, idx2freq_map, _ = get_word_map_from_file("poem.txt")
    else:
        print('Load prebuilt word map')
        word2idx_map, idx2freq_map = word_map
        
    
    total_words = sum(idx2freq_map)
    vocab_size = len(idx2freq_map)
    print("Model %s, optimizer: %s start" % (model_name, optimizer))
    print("Total Words: %d, Vocabulary: %d, Embedded to: %d-d vectors\n"
          % (total_words, vocab_size, embed_size))

    neg_table, keep_prob = create_sampling_table(idx2freq_map, sample=sample)
    parameters = init_parameters(vocab_size, embed_size, use_biases, init)
    
    if optimizer == 'gd':
        optimizer = GradientDescentOptimzer(learning_rate)
    elif optimizer == 'adam':
        optimizer = Adam(learning_rate, parameters)
    else:
        print('Unknown Optimzer!!')
        raise ValueError
        
    train_words = 0

    steps = 0
    avg_loss = 0.
    err_count = 0

    for epoch in range(1, n_epochs + 1):
        print("Epoch%d start" % epoch)
        with open(file_name, 'rb') as f:
        # train through sentence
            for line in f:
                # list of word's idx in sentence
                sen = sentence2index(line, word2idx_map, keep_prob)
                sen_set = set(sen)
                train_words += len(sen)

                for center, context in enumerate(sen):
                    # for words in window
                    for input_word in range(max(0, center - window),
                                            min(len(sen), center + window + 1)):
                        if input_word == center:
                            continue

                        target_words, labels = negative_sampling(context, sen_set, negative_samples, neg_table)
                        word_pairs = ([sen[input_word]], target_words)

                        parameters_proj = get_proj(parameters, word_pairs)

                        preds = forward_prop(parameters_proj)
                        loss = -np.log(abs(preds + labels - 1))

                        # gradient descent
                        grads_proj = back_prop(preds, labels, parameters_proj)
                        optimizer.update_proj(parameters, grads_proj, word_pairs)

                        avg_loss += np.sum(loss)
                        err_count += len(target_words)
                        steps += 1
                        if steps % 10000 == 0 and verbose >= 2:
                            print("Epoch%d, Trained Words: %d, Average Loss: %.4f, Time Cost: %.2f s" %
                                  (epoch, train_words, avg_loss / err_count, time.time() - start_time))
                            avg_loss = 0.
                            err_count = 0

            if verbose >= 1:
                print("Epoch%d finished, Trained Words: %d, Average Loss: %.4f, Time Cost: %.2f s" %
                      (epoch, train_words, avg_loss / err_count, time.time() - start_time))
                avg_loss = 0.
                err_count = 0

        # output to .json file
        save_embedding_to_file(model_name, parameters['embeddings'], epoch)
            
        print("Build file: ", model_name)
        print()

In [ ]:
train_embedding('try', 'poem.txt', (word2idx_map, idx2freq_map), learning_rate=0.025, optimizer='gd', verbose=2)

Load prebuilt word map
Model try, optimizer: gd start
Total Words: 2593895, Vocabulary: 5500, Embedded to: 30-d vectors

Epoch1 start
Epoch1, Trained Words: 3625, Average Loss: 0.6931, Time Cost: 1.32 s
Epoch1, Trained Words: 7194, Average Loss: 0.6926, Time Cost: 2.51 s
Epoch1, Trained Words: 10828, Average Loss: 0.6750, Time Cost: 3.69 s
Epoch1, Trained Words: 14437, Average Loss: 0.5634, Time Cost: 4.98 s
Epoch1, Trained Words: 17921, Average Loss: 0.4328, Time Cost: 6.23 s
Epoch1, Trained Words: 21810, Average Loss: 0.3178, Time Cost: 7.50 s
Epoch1, Trained Words: 25633, Average Loss: 0.2749, Time Cost: 8.74 s


In [9]:
embeddings = load_embedding_from_file('output/test/epoch2.json')

In [11]:
def get_top(word, n_top=5):

    norm = np.linalg.norm(embeddings, axis=0)
    if isinstance(word, str):
        idx = word2idx_map.get(word)
        vec = embeddings[:, idx]
        norm_w = norm[idx]
    else:
        idx = -1
        vec = word
        norm_w = np.linalg.norm(vec)

    product = np.dot(embeddings.T, vec)
    cosine = product / (norm * norm_w)

    ret = heapq.nlargest(n_top,
                         filter(lambda x : x[0] != idx,
                                [(x[0], x[1]) for x in enumerate(cosine)]),
                         key=lambda x : x[1])

    if isinstance(word, str):
        print(word)
    for x in ret:
        print(idx2word_map[x[0]], "%.3f" % x[1])

def get_vec(word, axis=1):
    return embeddings[:, word2idx_map.get(word)]

def get_similarity(w1, w2):
    if isinstance(w1, str):
        w1 = get_vec(w1)
    if isinstance(w2, str):
        w2 = get_vec(w2)
    norm1 = np.linalg.norm(w1)
    norm2 = np.linalg.norm(w2)
    return np.dot(w1, w2)/ (norm1 * norm2)

In [16]:
get_top(get_vec(u"大") - get_vec(u"小") + get_vec(u"慢"), 10)

大 0.710
干 0.689
怒 0.674
快 0.660
甯 0.659
籲 0.646
激 0.636
淳 0.635
禍 0.628
既 0.617


In [21]:
get_top(u"弓")
get_top(u"將")
get_top(u"歸")
get_top(u"微")
get_top(u"鬥")

弓
雕 0.845
鋒 0.826
刀 0.825
彎 0.825
珂 0.821
將
教 0.775
須 0.771
成 0.736
休 0.734
攻 0.732
歸
來 0.864
去 0.799
還 0.760
緱 0.746
游 0.745
微
霏 0.789
熏 0.697
暄 0.678
颸 0.674
濃 0.673
鬥
射 0.835
鬣 0.799
嚼 0.796
勒 0.790
鐙 0.781


In [76]:
def save_data_to_file(file_name, data):
    path = 'output/'
    if not os.path.isdir(path):
        os.makedirs(path)

    with open(path + file_name, "w") as f_out:
        f_out.write(json.dumps(data, indent=4))
        
def load_data_from_file(file_name):
    with open('output/' + file_name, 'r') as f:
        data = json.load(f)
    return data

In [77]:
save_data_to_file('word2idx_map', word2idx_map)
save_data_to_file('idx2word_map', idx2word_map)

In [72]:
word2idx_map = load_data_from_file('word2idx_map')
idx2word_map = load_data_from_file('idx2word_map')